In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd 

data = pd.read_csv('../input/fer2013/fer2013.csv') # load data from csv
data.head() # Gives first 5 rows

# Data Preprocessing

Beim Abrufen der Trainingsvariablen X und y aus Pixeln bzw. Emotionsspalten der CSV Datei werden diese in ein Numpy-Arrays konvertiert. Wir fügen unserem Feature-Vektor mithilfe der Funktion np.expand_dims () eine zusätzliche Dimension hinzu. Dies geschieht, um die Eingabe für unser CNN geeignet zu machen, das wir später entwerfen werden. Sowohl Funktionen als auch Beschriftungen werden als .npy-Dateien gespeichert, die später verwendet werden.

In [ ]:
import warnings
warnings.filterwarnings("ignore")

width, height = 48, 48

datapoints = data['pixels'].tolist()

#getting features for training
X = []
for xseq in datapoints:
    xx = [int(xp) for xp in xseq.split(' ')]
    xx = np.asarray(xx).reshape(width, height)
    X.append(xx.astype('float32'))

X = np.asarray(X)
X = np.expand_dims(X, -1)

#getting labels for training
y = pd.get_dummies(data['emotion']).to_numpy()

#storing them using numpy
np.save('fdataX', X)
np.save('flabels', y)

print("Preprocessing Done")
print("Number of Features: "+str(len(X[0])))
print("Number of Labels: "+ str(len(y[0])))
print("Number of examples in dataset:"+str(len(X)))
print("X,y stored in fdataX.npy and flabels.npy respectively")


In [ ]:
#Developing the Model

In [ ]:
import sys, os
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.losses import categorical_crossentropy
from keras.optimizers import Adam
from keras.regularizers import l2

num_features = 64
num_labels = 7
batch_size = 64
epochs = 100
width, height = 48, 48

x = np.load('./fdataX.npy')
y = np.load('./flabels.npy')

x -= np.mean(x, axis=0)
x /= np.std(x, axis=0)

#for xx in range(10):
#    plt.figure(xx)
#    plt.imshow(x[xx].reshape((48, 48)), interpolation='none', cmap='gray')
#plt.show()

#splitting into training, validation and testing data
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.1, random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.1, random_state=41)

#saving the test samples to be used later
np.save('modXtest', X_test)
np.save('modytest', y_test)

# Design und Ausführung des CNN Modell

Der Output zeigt uns  das sequentiell ausgeführte Modell. Ein sequentielles Modell ist ein linearer Stapel von Ebenen, bei dem Ebenen übereinander gelegt werden, während wir jedesmal von der Eingabeebene zur Ausgabeebene übergehen.

model.add (Conv2D ()) - Ist eine 2D-Faltung, die die Faltungsoperation wie am Anfang dieses Beitrags beschrieben ausführt.   Hier verwenden wir eine 3x3-Kernelgröße und eine gleichgerichtete lineare Einheit (ReLU) als Aktivierungsfunktion.

model.add (BatchNormalization ()) - Führt die Batch-Normalisierungsoperation für Eingaben in die nächste Ebene aus, sodass unsere Eingaben in einer bestimmten Skala von 0 bis 1 angezeigt werden, anstatt über den gesamten Ort verteilt zu sein.

model.add (MaxPooling2D ()) - Diese Funktion führt die Pooling-Operation für die Daten aus, wie am Anfang des Beitrags erläutert. Wir nehmen in diesem Modell ein Pooling-Fenster von 2x2 mit 2x2 Schritten.

model.add (Dropout ()) - Wie oben erläutert, ist Dropout eine Technik, bei der zufällig ausgewählte Neuronen während des Trainings ignoriert werden. Sie werden zufällig "abgebrochen". Dies reduziert das Overfitting.

model.add (Flatten ()) - Dies reduziert nur die Eingabe von ND auf 1D und hat keinen Einfluss auf die Stapelgröße.

model.add (Dense ()) - Gemäß Keras Documentation implementiert Dense die Operation: output = Aktivierung, wobei die Aktivierung die elementweise Aktivierungsfunktion ist, die als Aktivierungsargument übergeben wird. 

In [ ]:
#desinging the CNN
model = Sequential()

model.add(Conv2D(num_features, kernel_size=(3, 3), activation='relu', input_shape=(width, height, 1), data_format='channels_last', kernel_regularizer=l2(0.01)))
model.add(Conv2D(num_features, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Dropout(0.5))

model.add(Conv2D(2*num_features, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(2*num_features, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Dropout(0.5))

model.add(Conv2D(2*2*num_features, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(2*2*num_features, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Dropout(0.5))

model.add(Conv2D(2*2*2*num_features, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(2*2*2*num_features, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Dropout(0.5))

model.add(Flatten())

model.add(Dense(2*2*2*num_features, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(2*2*num_features, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(2*num_features, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(num_labels, activation='softmax'))

model.summary()

# Trainieren des Modells

#Dies ist ein ziemlich einfacher Codeabschnitt, bei dem zuerst das Modell mit categoryical_crossentropy als Loss-Function und mit der Verwendung des Adam-Optimierers kompiliert wird. Wir verwenden Accuracy als Metrik für die Validierung. Außerdem passen wir das Modell mit der festen Chargengröße (64 hier), den Epochen (hier 100) und den Validierungsdaten an, die wir durch Aufteilen der Trainingsdaten zuvor erhalten haben. 

In [ ]:
#Compliling the model with adam optimixer and categorical crossentropy loss
model.compile(loss=categorical_crossentropy,
              optimizer=Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-7),
              metrics=['accuracy'])

#training the model
model.fit(np.array(X_train), np.array(y_train),
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(np.array(X_valid), np.array(y_valid)),
          shuffle=True)

#saving the  model to be used later
fer_json = model.to_json()
with open("fer.json", "w") as json_file:
    json_file.write(fer_json)
model.save_weights("fer.h5")
print("Saved model to disk")